# Direct and inverse geometry of 2d robots

This notebook the main concept of kinematic tree, direct geometry and inverse geometry, but without the kinematic tree of Pinocchio. We only use the basic geometries of the our 3d viewer for displaying the simple robot that is used in this tutorial.

In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


## Set up
We will need NumPy, SciPy, and MeshCat Viewer for vizualizing the robot.
Scipy is a collection of scientific tools for Python. It contains, in particular, a set of optimizers that we are going to use for solving the inverse-geometry problem.

In [2]:
import time
import numpy as np
import pinocchio as pin
from utils.meshcat_viewer_wrapper import MeshcatVisualizer,translation2d,planar

<a id='section_display_objects'></a>
## Displaying objects
Let's first learn how to open a 3D viewer, in which we will build our simulator. We will use the viewer MeshCat which directly displays in a browser. Open it as follows:

In [3]:
viz = MeshcatVisualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


The following <viz> object is a client of the viewer, i.e. it will be use to pass display command to the viewer. The first commands are to create objects:

In [138]:
#ballID = 'world/ball'; viz.addSphere(ballID,.2,[1,0,0,1])
cylID1 = 'world/cyl1';   viz.addCylinder(cylID1,length=0.3,radius=.05,color=[0,0,1,1])
cylID2 = 'world/cyl2';   viz.addCylinder(cylID2,length=0.2,radius=.06,color=[1,1,0,1])
#boxID = 'world/box';   viz.addBox(boxID,[.5,.2,.4],[1,1,0,1])

In [139]:
frame1 = pin.SE3.Random()
frame2 = pin.SE3.Random()
frame3 = pin.SE3.Random()
frame4 = pin.SE3.Random()

viz.visualize_frame("frame1", frame1)
viz.visualize_frame("frame2", frame2)
viz.visualize_frame("frame3", frame3)
viz.visualize_frame("frame4", frame4)

In [140]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(frame1)))
viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(pin.SE3(frame1)))

In [141]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(frame2)))
viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(pin.SE3(frame2)))

In [142]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(frame3)))
viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(pin.SE3(frame3)))

In [143]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(frame4)))
viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(pin.SE3(frame4)))

In [144]:
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
viz.applyConfiguration(cylID2,pin.SE3ToXYZQUAT(pin.SE3(1)))

In a first time, we will work in 2D. Here is a shortcut to place an object from x,y,theta 2d placement, so-called *planar*. An example of a shorter positioning of a 2D object using this shortcut is:

In [145]:
viz.applyConfiguration(boxID,planar(0.1, 0.2, np.pi / 3))
viz.applyConfiguration(cylID,planar(0.1, 0.2, 5*np.pi / 6))

NameError: name 'boxID' is not defined

In [146]:
#viz.delete(ballID)
viz.delete(cylID1)
viz.delete(cylID2)
#viz.delete(boxID)

In [19]:
def screw_axis_q(transform):
    def skew(x):
        return np.array([[0, -x[2], x[1]],
                        [x[2], 0, -x[0]],
                        [-x[1], x[0], 0]])

    s_hat = pin.log(transform).angular / np.linalg.norm(pin.log(transform).angular)
    theta_dot = np.linalg.norm(pin.log(transform).angular)
    h = s_hat.T @ pin.log(transform).linear / theta_dot

    q = np.linalg.pinv(skew(pin.log(transform).angular)) @ (h * pin.log(transform).angular - pin.log(transform).linear)
    d = h * theta_dot
    return q, d

In [24]:
transform = pin.SE3.Random()
#transform = pin.XYZQUATToSE3(np.array([1,0,0,1,0,0,0]))

In [25]:
cylID1 = 'world/cyl1';   viz.addCylinder(cylID1,length=0.3,radius=.05,color=[0,0,1,1])
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
#transform = pin.SE3.Random()
viz.visualize_frame("transform", transform)
steps = 200

q, d = screw_axis_q(transform)
s_dot = pin.log(transform).angular / np.linalg.norm(pin.log(transform).angular)
viz.visualize_axis("screw_axis", q, s_dot, d)

s_hat = pin.log(transform)
for i in range(steps + 1):
    alpha = i / steps
    # Interpolate between identity (SE3.Identity()) and M using the exponential map of the weighted twist
    Mi = pin.exp(alpha * pin.log(transform))
    viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(Mi)))
    time.sleep(0.01)

In [ ]:
cylID1 = 'world/cyl1';   viz.addCylinder(cylID1,length=0.3,radius=.05,color=[0,0,1,1])
viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(1)))
#transform = pin.SE3.Random()
viz.visualize_frame("transform", transform)
steps = 200

q, d = screw_axis_q(transform)
s_dot = pin.log(transform).angular / np.linalg.norm(pin.log(transform).angular)
viz.visualize_axis("screw_axis", q, s_dot, d)

s_hat = pin.log(transform)
for i in range(steps + 1):
    alpha = i / steps
    # Interpolate between identity (SE3.Identity()) and M using the exponential map of the weighted twist
    Mi = pin.exp(alpha * pin.log(transform))
    viz.applyConfiguration(cylID1,pin.SE3ToXYZQUAT(pin.SE3(Mi)))
    time.sleep(0.01)